# scikit-learn 入門

本チュートリアルの主題であるディープラーニングの前に、一般的な機械学習アルゴリズムの実装方法を学びます。
モデル構築の手順や細かい調整など先に俯瞰して学んでおくことで、Chainer を使い始めた際の理解度が上がります。

Python で機械学習を扱う際には、一般的に **scikit-learn** がよく使用されるため、本章ではこの scikit-learn の使い方を含めて、機械学習によるモデル構築の過程を学んでいきましょう。

## scikit-learn（基礎編）

scikit-learn は Python のオープンソース機械学習ライブラリであり、分類や回帰などの様々なアルゴリズムが容易に実装できます。
すでに学んでいる NumPy の ndarray でデータのやり取りを行うことができ、これまで学んできたライブラリとの連携もしやすくなっています。

### scikit-learnが用意しているもの

本章は大きく以下の 6 つのセクションに分けられています。

- 回帰 (Regression)
- 分類 (Classification)
- クラスタリング (Clustering)
- 次元削減 (Dimensionality reduction)
- モデル選択 (Model selection)
- 前処理 (Preprocessing)

実装されている機械学習アルゴリズムや、その各種パラメータについては[公式のドキュメント](https://scikit-learn.org/stable/)に記載せれており、公式の情報はこちらで確認することができます。


### データセットの準備

前章で NumPy を用いて実装した、重回帰分析を scikit-learn を用いて実装していきます。  
データセットは前章で使用したものを再度使用します。  

In [1]:
import numpy as np

# xの定義
x = np.array([
    [2, 3],
    [2, 5],
    [3, 4],
    [5, 9],
])

print(x)

[[2 3]
 [2 5]
 [3 4]
 [5 9]]


切片を重みベクトルに含めて扱うため、デザイン行列の 0 列目に 1 という値を付け加えます。

In [2]:
# データ数（X.shape[0]) と同じ数だけ 1 が並んだ配列
ones = np.ones((x.shape[0], 1))

# concatenate を使い、1 次元目に 1 を付け加えていく
x = np.concatenate((np.ones((4, 1)), x), axis=1)

# 先頭に 1 が付け加わったデザイン行列
print(x)

[[1. 2. 3.]
 [1. 2. 5.]
 [1. 3. 4.]
 [1. 5. 9.]]


目標値として下記を使用します。

In [3]:
# t の定義
t = np.array([1, 5, 6, 8])

print(t)

[1 5 6 8]


### scikit-learnを用いた重回帰分析

scikit-learn が提供している重回帰分析を行うためのクラスを読み込み、今回用いるデータに適用していきます。  
詳細な使用方法は[公式のドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)を確認してください。  

scikit-learn がインストール済みであれば、`sklearn` という名前で `import` することができます。
Google Colaboratory では標準でインストールされているため、早速始めていきましょう。

In [4]:
import sklearn

scikit-learn では重回帰分析を行うためのクラスが `LinearRegression` という名前で定義されています。
scikit-learn ででこのクラスが定義されているモジュールは以下のような階層構造を持っています。

```
sklearn
├── linear_model
│   ├── LinearRegression
│   ├── ...
```

そこで、このような階層構造の下にあるクラスを呼び出す方法を 3 つ紹介します。

1 つ目は、`sklearn.linear_model` のように親モジュールを読み込むことです。

In [5]:
import sklearn.linear_model

In [6]:
model = sklearn.linear_model.LinearRegression()

2 つ目は `from` と`import` の 2 つを使って、親モジュールを読み込む方法です。

In [7]:
from sklearn import linear_model

In [8]:
model = linear_model.LinearRegression()

3 つ目は、2 つ目と同じく  `from` と`import` の 2 つを使って読み込む方法ですが、親モジュールを読み込むのではなく、対象のクラスを直接 `import` する方法です。

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
model = LinearRegression()

上記 3 つの方法のうち、どれを用いても構いませんが、コード中で `LinearRegression` クラスを参照するために書かなければならない文字列の長さが変わるため、状況に合わせて使い分けましょう。
今回はコードが最も短くなる 3 つ目の書き方を採用します。

初学者は、はじめに下記の 3 つのステップを覚えていきましょう。

1. モデルの定義：機械学習アルゴリズム選択し、そのアルゴリズムが実装されたクラスをインスタンス化します。
2. モデルの訓練：インスタンス内の属性としてもつパラメータを調整します。
3. 精度の検証：訓練済みモデルに対する精度の検証を行います。

重回帰分析のアルゴリズムは scikit-learn の中にクラスで定義されており、はじめにインスタンス化を行います。
クラスで定義した機能を利用するには、インスタンス化という操作が必要でした。今回はインスタンス化の際に呼び出されるイニシャライザ（`__init__`）は引数を取らないため、クラスの名前に続いて空の丸かっこ `()` を書いてインスタンス化を行っています。

In [11]:
# モデルの定義
model = LinearRegression()

一行でしたが、重回帰分析のモデルを定義することができました。

それでは、次のステップとして、モデルの訓練を行いましょう。
scikit-learn のクラスは訓練の際に `fit()` という関数名でインターフェースが統一されているので覚えておきましょう。
インターフェースが統一されているとは、重回帰分析以外の機械学習の手法でも同じ名前の関数名が使われているということです。
引数には、入力変数 `x` と教師データ `t` を与え、引数に使用する `x` や `t` は `numpy.ndarray` の形式が標準です。

In [12]:
# モデルの訓練
model.fit(x, t)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

訓練に関しても一行で書けてしまうため、実感が湧きにくいですが、これで完了です。

モデルの訓練が完了し、パラメータの値が調整されています。
それでは、パラメータを確認してみましょう。
重回帰分析では、重み `w` とバイアス `b` の２つがパラメータでした。
数式を使って説明する場合は、`b` を `w` で包括するように記述しましたが、scikit-learn では重みとバイアスが別の変数に格納されています。
重み `w` の確認には `model.coef_`、バイアス `b` の確認には `model.intercept_`をそれぞれ見てみましょう。

In [13]:
# 訓練後のパラメータ w
model.coef_

array([0.        , 0.71428571, 0.57142857])

In [14]:
# 訓練後のバイアス b
model.intercept_

-0.14285714285714057

今回は以前の章で紹介した NumPy での実装を参考にしたため、左の列を `1` で埋めていましたが、scikit-learn の `LinearRegression` では、変数としてバイアス `b`  が別で用意されているため、左の列を`1`で埋める必要がありません。

In [15]:
# xの定義
x = np.array([
    [2, 3],
    [2, 5],
    [3, 4],
    [5, 9],
])

In [16]:
# モデルの定義
model = LinearRegression()

In [17]:
# モデルの訓練
model.fit(x, t)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [18]:
# 訓練後のパラメータ w
model.coef_

array([0.71428571, 0.57142857])

In [19]:
# 訓練後のバイアス b
model.intercept_

-0.14285714285714235

このように、scikit-learn の `LinearRegression` を使用する限りは左の列をバイアスのために `1` で埋めるといった操作は不要であり、考慮すべき負担がひとつ減りました。

モデルの訓練が完了したら、精度の検証を行います。
検証も訓練と同じく `score` という関数名でインターフェースが統一されています。

In [20]:
# 精度の検証
model.score(x, t)

0.6923076923076923

さて、検証の結果、数値が返って来ましたが、この結果から善し悪しを測ることができなければ使えません。

scikit-learn のモデルが持つ `score` で返ってくる値（デフォルトの設定）の算出方法には、回帰と分類で異なる指標が使われています。

回帰の場合、**決定係数**と呼ばれる指標であり、計算式は以下のとおりです。

$$
R^{2} = 1 - \dfrac{\sum_{n=1}^{N_v}\left( t_{n} - y_{n} \right)^{2}}{\sum_{n=1}^{N_v}\left( t_{n} - \bar{t} \right)^{2}}
$$

ここで、検証に使うデータのサンプル数を $N_v$, $n$  番目のサンプルに対応する予測値を $y_{n}$, 教師データの平均値を $\bar{t}$ としています。

決定係数の最大値は 1 であり、もしこの値が 1 であれば（テストデータに対しては）完璧な予測ができていると言えます。
また、0 に近づくと予測としては良い結果とは言えません。
0 から 1 の間で判定することが一般的ですが、決定係数の定義としては負の値を取ることもあります。

### 訓練データとテストデータ

モデルの訓練と精度の検証に関する一連の流れを紹介しましたが、ここでひとつ問題があります。
その問題点は、今回モデルの訓練と精度の検証に用いるデータセットが同一であったことです。

ここで、ひとつ例を挙げて考えてみます。
例えば、受験の際に 10 年分の過去問を購入したとしましょう。
その手持ちの問題集の中で自分自身の実力を試したい場合に、以下のどちらが適切でしょうか。

- 10 年分の過去問を一通り学び、もう一度同じ 10 年分の過去問を実力テスト用に使用する
- 5 年分の過去問で学び、残りの答えを知らない 5 年分の過去問を実力テスト用に使用する

答えは後者です。
前者のように、答えを知っている問題を使用して実力テストを行ったとしても、本当の実力を測ることができません。

これは機械学習モデルの訓練と検証でも同じことが当てはまります。
実力をつけるために勉強する用の訓練データと、実力を測るためのテストデータは分けるべきであるということです。
しかし、上述した一連の流れでは、訓練と検証に同じデータセットを使ってしまっていました。

そこで、訓練データとテストデータを分けていきましょう。
訓練データとテストデータを分けることを**ホールドアウト法**と呼びます。
scikit-learn では、モデルの訓練や検証だけでなく、データセットの分割などを行う関数も用意されており、この関数を利用していきましょう。

In [21]:
# データセットを分割するモジュールの読み込み
from sklearn.model_selection import train_test_split

In [22]:
# 訓練データとテストデータの分割
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.3, random_state=0)

ここで、`train_test_split` の引数に `test_size=0.3` を与えています。
これはテストデータを全体の 30% と指定することを意味しています。
自動的に訓練データは残りの 70% となります。
また、`random_state` という引数が指定されていることにも注意が必要です。

`train_test_split` では前半  70% と後半 30% のようにデータセットを分割するのではなく、全サンプルの中からランダムに 70% を訓練データとして抽出し、残った 30% をテストデータとして用います。
例えば、データセット中のサンプルが、目標値が 1 のサンプルが 10 個、2 のサンプルが 8 個、3 のサンプルが 12個…というように、カテゴリごとにまとめられて並んでいることがあります。
その際に、例えばこのデータセットの先頭から 18 個目のところで訓練データとテストデータに分割すると、訓練データには目標値が 3 のデータが 1 つも含まれなくなってしまいます。

そこで、ランダムにデータセットを分割する方法が採用されています。
NumPy の章でも紹介した通り、乱数を扱う際には再現性が重要であり、その再現性を確保するために、乱数のシードを `random_state` という引数で指定し、固定しています。

それでは、分割後の訓練データを用いてモデルの訓練、精度の検証を行いましょう。

In [23]:
# モデルの訓練（訓練データ）
model.fit(x_train, t_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [24]:
# 精度の検証（テストデータ）
model.score(x_test, t_test)

-15.999999999999982

今回はサンプル数が非常に少ないため、テストデータでの予測精度が低く、決定係数の値が負の値を取るような悪い結果となっています。
つまり、訓練データで構築したモデルがテストデータに対してうまく当てはまっていないことがわかります。

ここで、精度の検証では、訓練データに対する結果も確認しておくことをおすすめします。
選択した機械学習アルゴリズムのモデルが訓練データに対してうまく当てはまっていないか、訓練データ自体にはモデルがうまく当てはまっていますが、テストデータに対して訓練済みモデルへの当てはまりが悪いかを判断することができます。
この原因を切り分けることで、それぞれ必要な対応を変えることができます。

In [25]:
# 精度の検証（訓練データ）
model.score(x_train, t_train)

1.0

訓練データに対する決定係数は 1 となっており、精度が非常に高く予測できていると言えます。
一方で、テストデータに対して精度が低く、このような状況を**過学習 (overfitting)** と呼ばれます。
この過学習への対策は scikit-learn の応用編にて解説します。

### 推論

訓練済みモデルに新たな入力変数を与えることによって、予測値を求めます。
scikit-learn では `predict` として関数が定義されています。

ひとつ注意すべき点として、推論を行う際の入力となるデータは、訓練時と同様に行列の形である `(サンプル, 入力変数)` を格納して渡す必要があります。

In [26]:
# 新たな入力x_newの定義
x_new = np.array([[2, 3]])

ここで、`[2, 3]` とすると、行列の形が `(入力変数)` だけとなり、上述の `(サンプル, 入力変数)` という行列の形になりません。
そこで、`[[2,3]]` とすることで 1 サンプルであることを明記できます。

In [27]:
# 予測値の計算
y = model.predict(x_new)
y

array([1.])

## scikit-learn（応用編）

この節では scikit-learn を用いて、下記の内容について学びます。  

- scikit-learn で用意されているデータセットを使用
- 重回帰分析以外のアルゴリズムでの実装 
- 前処理
- パイプライン化
- ハイパーパラメータの調整

ここで、**前処理**とは、モデルの訓練を行う前に、データセットに対して外れ値除去などの処理を施すことです。
外れ値除去以外にも、入力変数を必要なものだけ使用する**変数選択**や、入力変数に対して $\log$ などの変換を施す**変数変換**、入力変数のスケーリングを行う**正規化**などが代表的です。

また、もうひとつ初出である**ハイパーパラメータ**は機械学習では頻出する単語です。
重回帰分析では、重み `w` やバイアス `b` のような訓練データによって最適化を行う変数を**パラメータ**と呼んでいました。
それに対して、訓練データを用いて最適化を行わないけれど、調整が必要な変数のことを**ハイパーパラメータ** と呼びます。
各機械学習アルゴリズムに固有のハイパーパラメータがあります。
重回帰分析はハイパーパラメータをアルゴリズム内で持っていませんが、たとえば前処理としてスケーリングを行うか否かは一種のハイパーパラメータと言えます。
この節の後半で、具体的なハイパーパラメータとその調整法について紹介していきます。

### scikit-learn で用意されているデータセットを使用

#### データセットの読み込み

まずはじめに、例題で使用するデータセットに関してその内容を紹介します。
scikit-learn では、デモ用としていくつかのデータセットが準備されています。
今回はその中から、米国ボストン市郊外における地域別の物件価格のデータセットを使用することとします。

このデータセットには 506 件のサンプルが存在し、各サンプルには対象地域の平均物件価格と、それに紐づく情報として対象地域の平均的な物件情報、人口統計情報、生活環境に関する情報などが含まれています。  
このデータセットを用いて、物件や人口統計などの情報を入力変数として、目標値である平均物件価格を予測するモデルを構築します。
入力変数は全部で 13 種類あり、詳細は以下の通りです。

- CRIM : 人口 1 人あたりの犯罪発生率
- ZN : 25,000 平方フィート以上の住宅区画が占める割合
- INDUS : 非小売業が占める面積の割合
- CHAS : チャールズ川に関するダミー変数 (1 : 川沿い，0 : それ以外)
- NOX : 窒素酸化物の濃度
- RM : 住居あたりの平均部屋数
- AGE : 1940 年以前に建てられた物件の割合
- DIS : 5 つのボストン雇用施設からの重み付き距離
- RAD : 都心部の幹線道路へのアクセス指数
- TAX : $ 10,000 あたりの固定資産税の割合
- PTRATIO : 教師 1 人あたりの生徒数
- B : 黒人の比率を表す指数
- LSTAT : 低所得者の割合

ここで、**ダミー変数**とは、数値ではないデータを数値に変換することで、`CHAS` では川沿いに位置するか否かを `0` もしくは `1` の数値に置き換えています。

それでは、あらかじめ scikit-learn で用意されている関数を用いて、データセットを読み込みましょう。

In [28]:
from sklearn.datasets import load_boston

In [29]:
dataset = load_boston()

変数 `dataset` の `data` という属性に入力変数、`target` という属性に目標値が格納されています。

In [30]:
x = dataset.data
t = dataset.target

In [31]:
# 入力変数xの確認（先頭の３件）
x[:3]

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
        6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02,
        1.5300e+01, 3.9690e+02, 4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9690e+02, 9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9283e+02, 4.0300e+00]])

In [32]:
# xの形の確認（行, 列）
x.shape

(506, 13)

In [33]:
# 目的値tの確認（先頭の３件）
t[:3]

array([24. , 21.6, 34.7])

In [34]:
# tの形の確認
t.shape

(506,)

#### 訓練データ（Train）、テストデータ（Test）に分割

前述したホールドアウト法を使用して検証を行うため、訓練データとテストデータに分割していきます。

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
x_train, x_test, t_train, t_test = train_test_split(x, t, train_size=0.7, random_state=0)

#### 重回帰分析

新しいアルゴリズムを試す前に、本章の前半で学んだ重回帰分析をまず適用してみましょう。

In [37]:
# モデルの定義
from sklearn.linear_model import LinearRegression
model = LinearRegression() 

In [38]:
# モデルの訓練
model.fit(x_train, t_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

ホールドアウト法では訓練データによりモデルの訓練を行うことに再度、気をつけましょう。

訓練データとテストデータに対して、精度の検証を行います。

In [39]:
# モデルの検証（訓練データ）
model.score(x_train, t_train)

0.7905724291908702

In [40]:
# モデルの検証（テストデータ）
model.score(x_test, t_test)

0.6671893494370862

この結果より、今回は特に大きなオーバーフィッティングは起こしていないことがわかります。

補足として、モデルが訓練データに対してすら良い精度で予測できない状態を**未学習 (underfitting)** といいます。  
アンダーフィッティングが起きている場合、現状の機械学習アルゴリズムがデータの特徴を捉えるには不十分である可能性があります。
その場合は、アルゴリズムの変更や、入力データの特徴をより適切に表現できるような変換を導入するなどして、改善を試みます。  

逆にオーバーフィッティング（過学習）の場合、アルゴリズムでデータの特徴をある程度捉えられていることは確認できているので、モデルが過学習しないように対策していきます。  
代表的な方法としては、前述した**ハイパーパラメータ**を調整していくことで解決できる場合があります。  
具体的な調整についてはこれから紹介します。

このように、望ましい結果が得られないといっても、それぞれの状況を把握することで次に打つべき対策が変わってくるため、訓練データとテストデータの両方に対する検証を行うことは重要であることが分かります。

### 重回帰分析以外のアルゴリズムでの実装

**サポートベクターマシン (SVM: Support Vector Machine)** は実務でもよく使われる機械学習アルゴリズムのひとつです。
背景の数学の詳細な解説はここでは省略しますが、下図のように入力変数と目標値の間の関係が**線形**でないことがあります。
線形という新しい用語が登場し、厳密性には欠けますが、ひとまず入出力間の関係が直線で表現できることを線形と言うのだと捉えてください。
単回帰分析を含めた重回帰分析では、入出力間の関係が線形な場合しかうまく関係性を捉えることができません。
特徴量として $x^{2}$ や $\sin {x}$ など非線形な関数を採用することも考えられますが、複数の特徴量に対しての組み合わせは無限に存在し、その背景にある物理的な現象を正しく理解できている状況以外、現実的な選択とは言えません。

そこで、SVM では、**特徴関数** $\phi$ という**特徴空間へ変換**する関数を用います。
入力変数 $x_{i}$ を特徴空間へ非線形変換 $\phi(x_{i})$し、変換後の特徴空間において線形回帰を行うという工夫を行います。
実際には入力変数を変換する特徴関数を明示的に決定せずに、特徴空間での内積（2 つの入力変数の特徴空間上での距離に相当）を直接求めることができる**カーネル関数** $k$ を用いるため、このアプローチは**カーネルトリック**と呼ばれます。

![SVRイメージ](images/06/06_01.png)

SVM は数学的に重回帰分析よりも遥かに難易度が上がりますが、scikit-learn では重回帰分析と同様に、あらかじめ用意された関数を使っていくだけでとても簡単に実装できます。
scikit-learn での SVM の詳細についてはこちらの[公式ドキュメント](https://scikit-learn.org/stable/modules/svm.html#svr)が便利です。

SVM は回帰と分類の両方に対応しており、回帰の場合は **SVR (Support Vector Regression)**, 分類の場合は **SVC (Support Vector Classification)** という名前で scikit-learn 内では定義されています。

In [41]:
# モデルの定義
from sklearn.svm import SVR
model = SVR() 

In [42]:
# モデルの訓練
model.fit(x_train, t_train) # 訓練データを使ってモデルの学習

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

ここで、ひとつ注意したいのは、ここでは `SVR` を引数なしでインスタンス化しているため、すべての引数に対して `C=1.0` や `epsilon=0.1` のようなデフォルトの値が設定されていることです。
最初はこれらの値を指定することなく進めますが、当然これらの値にも意味が存在しています。

最適化手法を使用して調整する変数を**パラメータ**と呼び、上述の `C` のように機械学習アルゴリズムの挙動を制御するパラメータのことを**ハイパーパラメータ**と呼びます。
`fit` の関数ではパラメータを調整することができますが、その挙動を制御するハイパーパラメータの調整まで行うことはできません。
ハイパーパラメータの調整に関しては後述します。

まずはデフォルトのハイパーパラメータを用いて、パラメータの最適化を行った場合の精度を検証してみましょう。

In [43]:
# 精度の検証（訓練データ）
model.score(x_train, t_train)

0.1228199964193637

In [44]:
# 精度の検証（テストデータ）
model.score(x_test, t_test)

0.0003209859005506299

この結果より、訓練データとテストデータともに重回帰分析よりも精度が低い結果となりました。
本来、線形回帰の手法の問題点を解決しているはずの SVM でしたが、望ましい精度が得られていません。

この問題点として、以下の 2 つをよく考えます。

- **前処理**を適切に行えているか
- **ハイパーパラメータ**が適切に調整できているか

### 前処理

**前処理 (preprocessing)** とは、欠損値や外れ値の除去・補完から、変数変換、特徴量選択、正規化といった処理を施すことを指します。
それぞれのアルゴリズムに合わせた前処理が必要となり、そのためにアルゴリズムの特性を知っておく必要があります。

それでは、SVR に対する精度向上のための前処理を考えていきましょう。
SVR ではカーネルトリックの際に**距離**を使用することが一般的であり、基礎数学の章で解説したとおり、入力変数間のスケールが統一されていない場合には大きなスケールの変数に影響されてしまいます。
**距離**をアルゴリズム内で使用する場合、**正規化**と呼ばれるスケールを統一する処理を施すことがこの問題の対策として考えれます。
ディープラーニングでも同様に正規化を施す場合があります。[<sup>*2</sup>](#fn2)  。

<span id="fn2"><sup>*2</sup>：<small>ディープラーニングでよく用いられる正規化は[Batch Normalization](Batch Normalizationのリンクの挿入)があります。</small></span>

前処理に関しても、scikit-learn にいくつかの関数があらかじめ用意されています。
正規化にも、平均 0, 標準偏差 1 に変換する `StandardScaler` と、最小値 0, 最大値 1 に変換する `MinMaxScaler` があります。
今回は `StandardScaler` を使用していきましょう。

In [45]:
from sklearn.preprocessing import StandardScaler

モデルの定義と同様に、インスタンス化を行います。

In [46]:
scaler = StandardScaler()

この `scaler` では、正規化を行うために、平均と標準偏差の値が必要となります。
この値を算出するために、モデルの場合と同様に `fit` を用います。
このときに注意すべき点として、すべてのサンプルではなく、訓練データを用いてこれらの値を算出することです。

In [47]:
scaler.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

`fit` によって算出された値が scaler の属性として格納されていることが確認できます。

In [48]:
# 平均
scaler.mean_

array([3.27599073e+00, 1.20231788e+01, 1.10447682e+01, 6.62251656e-02,
       5.56298013e-01, 6.33063576e+00, 6.96927152e+01, 3.66313642e+00,
       9.83443709e+00, 4.10609272e+02, 1.84635762e+01, 3.65675828e+02,
       1.27405298e+01])

In [49]:
# 分散
scaler.var_

array([3.98109942e+01, 5.74266019e+02, 5.40302117e+01, 6.18393930e-02,
       1.35768756e-02, 5.04099265e-01, 8.20035775e+02, 3.58008203e+00,
       8.16480856e+01, 3.33497480e+04, 4.80112363e+00, 6.30709364e+03,
       5.53191295e+01])

標準偏差ではなく分散が属性として格納されていますが、これは分散が 0 であった場合の対応などを変換する際の関数で定義しているためですが、そこまでの詳細は現時点で気にする必要はありません（[参考: github](https://github.com/automl/paramsklearn/blob/master/ParamSklearn/implementations/StandardScaler.py)）。

正規化を施す際には `transform` を使用します。

In [50]:
x_train_scaled = scaler.transform(x_train)
x_test_scaled  = scaler.transform(x_test)

In [51]:
# モデルの訓練
model.fit(x_train_scaled, t_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [52]:
# 精度の検証（訓練データ）
model.score(x_train_scaled, t_train)

0.5297457196085098

In [53]:
# 精度の検証（テストデータ）
model.score(x_test_scaled, t_test)

0.5041470644131514

この結果より、正規化を事前に施すことにより、精度を大幅に向上させることができました。
重回帰分析の方が精度が高かったため、まだもう一段階工夫が必要となりますが、前処理の有効性が示せました。

### パイプライン化

前処理用の `scaler` と SVR の `model` をそれぞれに訓練していましたが、scikit-learn にはパイプラインと呼ばれる一連の処理を統合できる機能があります。
前述の処理をまとめていきましょう。

In [54]:
from sklearn.pipeline import Pipeline

In [55]:
# パイプラインの作成 (scaler -> svr)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR())
])

In [56]:
# 一連の流れでモデルの訓練
pipeline.fit(x_train, t_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svr', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False))])

In [57]:
# モデルの検証（訓練データ）
pipeline.score(x_train, t_train)

0.5297457196085098

In [58]:
# モデルの検証（テストデータ）
pipeline.score(x_test, t_test)

0.5041470644131514

このようにパイプライン化させることで、`x_train_scaled` のような間の変数を挟むことが必要なくなりました。
テストデータの正規化を忘れてしまうことがよくあるため、人的ミスを防ぐためにもパイプライン化は有効な手段といえます。

### ハイパーパラメータの調整

前述したとおり、SVM などアルゴリズムを制御するためのパラメータをハイパーパラメータと呼びます。
ハイパーパラメータは `fit` の関数では訓練することはできません。

そこで、**グリッドサーチ**と呼ばれる方法では、下図のようにハイパーパラメータ (ex. $C$, $\gamma$) の候補を格子状に切り、それぞれの値で確かめていく方法が有効です。
この方法以外にも、**ランダムサーチ**や**ベイズ最適化**を用いた方法もあるため、興味のある人は調べてみてください。
今回は、グリッドサーチによる実装を紹介していきます。

![](images/06/06_02.png)

訓練データは目的関数を最適化するようにパラメータを調整しますが、ハイパーパラメータを調整するためのデータではありません。
そのため、ハイパーパラメータを調整するためのデータも別で必要となり、これを**検証データ (validation data)** と呼びます。
つまり、ハイパーパラメータを持つアルゴリズムの場合、訓練データ・検証データ・テストデータの 3 つをそれぞれ用意します。

特に、ハイパーパラメータの調整を行う場合には **交差検証 (Cross Validation)** と呼ばれる、下記の図のようにデータを交差させる方法がよく用いられ、**$K$-分割交差検証** と呼ばれます。
下図は $K=5$ の場合であり、もともと訓練データであったものを 5 分割し、4 つは訓練データとしてパラメータを調整し、残りの1 つを検証データとして目的関数の値を算出します。
これをすべての組み合わせで行い、最終的にその算出された目的関数の平均を検証結果の値とします。
訓練データの全サンプルを検証用にも使用できるため、検証用データが恣意的にもしくは恣意的でなくても偏ってしまう問題を解決できます。

また、データセットのサンプル数が少ない場合にも有効です。
訓練データとテストデータを分割して訓練データが減ってしまい、さらに訓練データを検証データと分割すると、パラメータの調整に使用できるサンプル数が極端に少ない状況が考えられます。
$K$-分割交差検証であれば、すべての訓練データを訓練と検証に用いることができます。
また、検証データのサンプル数を 1 とすることもあり、訓練データのサンプル数を $N_t$ とすると、$N_{t} -1$ サンプルでモデルを訓練し、1 サンプルで検証を行います。
これを $N_t$ 回繰り返し、その平均を算出します。
このように、検証データのサンプル数を 1 とする場合は、**Leave-one-out 交差検証 (LOOCV)** と呼ばれます。

![](images/06/06_01.png)

交差検証で求めた目的関数の値をグリッドサーチの指標として利用することで、最良なハイパーパラメータの組み合わせを見つけることができます。
これを scikit-learn では `GridSearchCV` として準備されており、手軽にハイパーパラメータの調整を行うことができます。

In [59]:
from sklearn.model_selection import GridSearchCV

In [60]:
# グリッドサーチを行うハイパーパラメータの候補を定義
# Pipelineを使う場合は <処理の名前>__<パラメータ名>: [...]
# Pipelineを使わない場合は <パラメータ名>: [...]
params = [
    {'svr__C': [1, 10, 100, 1000], 'svr__gamma': [0.001, 0.01, 0.1, 1]}
]

モデルとしては、前回パイプライン化を行った前処理を含めた SVR である `pipeline` を使用し、`estimator` という引数に指定します。
ハイパーパラメータの候補は `param_grid` に、分割の数 $K$ は 5 として、`cv` という引数に指定します。
また、回帰の際に見落としがちですが、目的関数を指定する `scoring` を `neg_mean_squared_error` としておきましょう。
`neg` は negative のことであり、本来は `mean_squared_error` を使用するところですが、scikit-learn の `GridSearchCV` では最大化を行うようにアルゴリズムが実装されています。
そのため、最大化を行うアルゴリズムで最小化を行うために、`neg` が付いた目的関数を使用しています。

In [61]:
# 交差検証 + グリッドサーチを行うモデルを定義
model_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=params,
    cv=5,
    scoring='neg_mean_squared_error'
)

In [62]:
# モデルの訓練
model_grid.fit(x_train, t_train)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svr', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'svr__C': [1, 10, 100, 1000], 'svr__gamma': [0.001, 0.01, 0.1, 1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

グリッドサーチも含めた訓練が終わると、最終的に最も良かったパラメータの組み合わせを `best_params_` で確認することができます。

In [63]:
model_grid.best_params_

{'svr__C': 1000, 'svr__gamma': 0.01}

そして、最良なハイパーパラメータの組み合わせを持つモデルが `best_estimator` で取り出すことができます。

In [64]:
model = model_grid.best_estimator_

最後に、このハイパーパラメータも調整も行ったモデルでの精度も検証しましょう。

In [65]:
# 精度の検証（訓練データ）
model.score(x_train, t_train)

0.9560597770245561

In [66]:
# 精度の検証（テストデータ）
model.score(x_test, t_test)

0.7618184466251714

この結果より、ハイパーパラメータの調整によりさらに予測精度を向上させることができ、その調整による有効性を示せました。
データの前処理とハイパーパラメータの調整は基本的にいつも行うため、覚えておきましょう。
ハイパーパラメータの名前や値の候補の与え方には数学的な背景の理解や経験が必要となります。
scikit-learn の公式ページに[グリッドサーチによるハイパーパラメータの調整](https://scikit-learn.org/stable/modules/grid_search.html)に関する例が紹介されているほか、最近では [Optuna](https://github.com/pfnet/optuna/tree/master/examples) というハイパーパラメータを最適化するためのフレームワークも登場しています。

scikit-learn ではその他にも**決定木 (decision tree)** や **主成分分析 (principal component analysis)** など様々な手法が既に実装されています。
使い方は今回の一連の手順とほとんど同じであるため、色々な方法を試してみましょう。
用意されているアルゴリズムは[公式ページ](https://scikit-learn.org/stable/)から確認することができます。